# FIPS Dataset class

This notebook illustrates basic functionality with the `FIPSDataset` class that holds FIPS 140 dataset

In [9]:
from sec_certs.dataset.fips import FIPSDataset
from sec_certs.sample import FIPSCertificate

## Get fresh dataset snapshot from mirror

In [ ]:
dset: FIPSDataset = FIPSDataset.from_web_latest()
print(len(dset))

## Do some basic dataset serialization

In [ ]:
# Dump dataset into json and load it back
dset.to_json("./fips_dataset.json")
new_dset = FIPSDataset.from_json("./fips_dataset.json")
assert dset == new_dset

## Simple dataset manipulation

In [8]:
# Get certificates from a single manufacturer
cisco_certs = [cert for cert in dset if "Cisco" in cert.web_scan.vendor]

# Get certificates with some CVE
vulnerable_certs = [cert for cert in dset if cert.heuristics.related_cves]

# Show CVE ids of some vulnerable certificate
print(f"{vulnerable_certs[0].heuristics.related_cves=}")

## Dissect single certificate

In [ ]:
# Select a certificate and print some attributes
cert: FIPSCertificate = dset["542cacae1d41132a"]

print(f"{cert.web_scan.module_name=}")
print(f"{cert.heuristics.cpe_matches=}")
print(f"{cert.web_scan.level=}")

## Serialize single certificate

In [13]:
cert.to_json("./cert.json")
new_cert: FIPSCertificate = FIPSCertificate.from_json("./cert.json")
assert new_cert == cert

## Create new dataset and fully process it

*Warning*: It's not good idea to run this from notebook. It may take several hours to finnish. We recommend using `from_web_latest()` or turning this into a Python script.

In [ ]:
dset = FIPSDataset()
dset.get_certs_from_web()
dset.convert_all_pdfs()
dset.pdf_scan()
dset.extract_certs_from_tables()
dset.finalize_results()

# TODO: Resolve https://github.com/crocs-muni/sec-certs/issues/210 and refactor this
# if not no_download_algs:
#     aset.get_certs_from_web()
#     logger.info(f"Finished parsing. Have algorithm dataset with {len(aset)} algorithm numbers.")
#     dset.algorithms = aset

# TODO: Resolve https://github.com/crocs-muni/sec-certs/issues/211 and uncomment
# dset.plot_graphs(show=False)